# Local Python iRIGS
#### Author: Colleen Xu
Winter - Spring 2020

## Intro

This is a Python3 Jupyter-notebook implementation of iRIGS (Integrative framework for Risk Gene Selector). **The notebook explains how the algorithms are implemented in detail, and it can be used to start modifying iRIGS for other studies/use cases/input data.** The [original R code](https://www.vumc.org/cgg/irigs) was released by the Computational Genetics and Genomics Lab at Vanderbilt University as part of the [*Nature Methods* paper](https://www.ncbi.nlm.nih.gov/pubmed/30988527):  

Wang Q, Chen R, Cheng F, Wei Q, Ji Y, Yang H, et al. A Bayesian framework that integrates multi-omics data and gene networks predicts risk genes from schizophrenia GWAS data. Nature Neuroscience. 2019 May;22(5):691–9.  

I relied heavily on the methods of this paper and its [Supplementary Note](https://www.nature.com/articles/s41593-019-0382-7#Sec26). **Below, I assume that the user of this notebook has downloaded the data from https://www.vumc.org/cgg/irigs and the notebook is in the same level/directory as the supporting_files directory and SNP_file directory.**      

**My implementation differs from the original code in several ways. All major changes are noted and explained in detail throughout the notebook. A brief description of changes:**   
* Different language (Python vs R)
* Preprocessing the input gene information differently and before picking genes from flanking regions around GWAS SNPs   
* Treating missing values for numbers of enhancers the same (replaced with zeroes)  
* Gibbs sampling is not done on SNP loci with only one candidate gene.  
* Reproducing the *Nature Methods* paper's Supplementary Table 1 as output  

**Hanging Questions (not yet addressed):**  
* There is a matrix included in the supporting files (go_propogation_probality_rp_0.3.RData). Is this matrix the transition probabilities, calculated from running random walk with restart until the stopping threshold was reached? Or is this the normalized weighted adjacency matrix for the GO-annotation-edge-weight network? 
* A normal distribution is assumed for the transformed variables created by the Mahalanobis transformation. However, my qq plots and normality tests on the variables fail to show that the variables are normally distributed. Would one proceed with using the normal distribution to get p-values (after all, the transformation is supposed to create variables that follow a normal distribution)? Or would one look to see if another distribution is a better fit?
* The probability of a gene having a de novo mutation associated with schizophrenia (used as a p-value) is calculated  using a hard-coded denominator of 20,000. How was this number chosen (is it the assumed number of genes in the genome)?
* What is the rationale for getting the negative log of the combined p-values (from Fisher's method) (extra_evi.R line 187)? I assumed that it was because the negative log would be a continuous measure of information, where a greater number would indicate more evidence that the candidate gene was a risk gene. (ref: https://lesslikely.com/statistics/s-values/)
* I am confused by extra_evi.R line 202, where it appears that only one surrogate Bayes factor is kept per gene name. What is the rationale for this? My thinking is that since the surrogate Bayes factor includes "distance to SNP" as input, the factor differs depending on which SNP a given gene is associated with. I have therefore kept all the calculated surrogate Bayes factors in my implementation of iRIGS; during Gibbs sampling, the Bayes factors specific to the sampled SNP locus and candidate genes are pulled.

### Description of iRIGS

This framework was designed to find the genes "most likely" to be disease risk genes. Starting with schizophrenia (SCZ) GWAS data, the genes neighboring the SNPs (loci) are found. During Gibbs sampling, the candidate genes for a SNP loci are weighed based on:  

* their "closeness" to the other sampled genes (around other SNP loci) in a gene-gene network made by the authors   
  - with the code, the iRIGS authors provided what I believe is the transition probability matrix obtained by running the random walk with restart (RWR) algorithm on their original network. Their original network was dense, with edge weights between genes based on shared GO annotations. {? See the "Hanging questions" section above: Is this the transition probabilities from the RWR or just the normalized weighted adjacency matrix for the original network? We are missing code to generate this matrix and explore the original network.}  
* their genomic distance from the SNP (distance from gene transcription start site (TSS) to SNP in genomic coordinates): abbreviated DTS (distance from transcription start)  
  - based on gene information file provided by iRIGS authors {? We are missing metadata on where it was downloaded from and when}   
* the number of enhancers (predicted distal regulatory element (DRE) - promoter links) they have  
  - Brain HiC data from [the developing cerebral cortex - cortical/subcortical plate (CP) and germinal zone (GZ), Supplementary Tables 23 and 24](https://www.nature.com/articles/nature19847#Sec31): tsv versions provided by iRIGs authors     
  - CapHiC data from the cell line GM12878 (not specific to brain): processed data file provided by iRIGs authors {? We are missing code on how to generate this file from "raw data files" from ArrayExpress}           
  - FANTOM5 data: data file provided by iRIGs authors {? We are missing metadata on where it was downloaded from and when. I'm not sure if this includes all tissues or not.}   
* de novo mutation (DNM) data for schizophrenia from literature: processed data files provided by iRIGS authors {? We are missing code on how to generate this file}   
* differential expression (DE) data comparing schizophrenia vs control cases from the Commonmind consortium: data file provided by iRIGS authors {? It is unclear if this file was made manually or with a code pipeline}       

**Assumptions:**  
* It is assumed that a gene with more "incoming regulatory links" (enhancers) is more likely to be a disease gene (discussed in the original paper).  
* All genomic coordinates are in hg19 (GRCh37).  

## Import packages

This code uses numpy, pandas, and scipy. It is designed to run simply with as few dependencies as possible. Some packages/tools are included for analyzing performance 

In [1]:
import numpy as np
import pandas as pd
from scipy import stats  ## use for pvalues, chidist

from pathlib import Path  ## for dealing with path names regardless of machine (Windows/Mac/Linux)
import math 
## get from https://anaconda.org/conda-forge/pyreadr
import pyreadr  ## import RData objects! 

import re  ## for regular expressions
import gc ## to delete/clean up dataframes I'm not using anymore
import time  ## to time long code chunks 

## for printing every line that will give output
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
## don't run unless you want to check your environment
# !conda info --envs  ## check what environment I am in
# !conda list ## check what packages I have

## checking if line profiler is working: https://github.com/rkern/line_profiler
# %lprun?

### Notes on using pandas

#### Using square brackets [ ] (the indexing operator) with pandas DataFrames:
The behavior will be different depending on the object you put in the square brackets. [Reference](https://medium.com/dunder-data/selecting-subsets-of-data-in-pandas-39e811c81a0c)

* string: return a column as a Series
* list of strings: return all the columns named, in order, as a DataFrame
* a slice of index labels or integers: select rows
* **a sequence of booleans: select all rows where True.** Can be a list, tuple, pandas Series, numpy array. 
    - For a pandas Series, the index exactly match the DataFrame's index. 
    - For numpy arrays, it must be the same length as the DataFrame. 

I will be using this last option (a sequence of booleans) often. Because of the constraints, these sequences are almost always made from the data (DataFrame) by using a logical expression on the data. 

#### Using multiple conditions in Boolean indexing
* Pandas will not accept the Python keywords (and, or, not). It will accept & (and), | (or), and ~ (not). 
* The multiple conditions can be written in one line inside the [ ]...but every condition **must be in parentheses** 
* consider breaking complicated indexing into multiple steps, saving the boolean sequences as variables

#### Building a dataframe with a for loop (bind rows, append, concat):
* It is more efficient to build a list of rows as iterables (tuples/dictionaries/lists), then turn this list into a dataframe. [Reference](https://stackoverflow.com/a/56746204)
* Try not to iterate over a DataFrame when you can. [Reference](https://engineering.upside.com/a-beginners-guide-to-optimizing-pandas-code-for-speed-c09ef2c6a4d6)

## Find candidate genes for each SNP 

**CHANGES from original code:** 
* I am filtering the human gene list BEFORE finding candidate genes for SNPs; this should make the search for candidate genes more efficient.  
* I am filtering differently: original code relies heavily on the "official_name" column, but I will be working with ENSG IDs ("Name" column) and the "alias_symbol" column instead. There is an EDA code chunk showing the reasons for the change.   
* I chose to drop certain genes because they were completely unconnected in the network. With the current framework design, these genes end up with a "network" factor of 0 and would not be chosen during the random sampling (unless all candidate genes for a SNP were in this situation - then a uniform probability of picking a gene would be implemented).  
**Reminder: I assume that the user of this notebook has downloaded the data from https://www.vumc.org/cgg/irigs and the notebook is in the same level/directory as the supporting_files directory and SNP_file directory.**    

### Load gene info, gene-gene network transition probabilities 

In [3]:
gene_info_path = Path.cwd().joinpath('supporting_files', 'All_human_genes')
gene_info = pd.read_csv(gene_info_path, sep = '\t', header = 0)  
gene_info.shape  ## 53934 rows, 13 columns

## takes around 30 seconds to load gene-gene network matrix
network_matrix_path = Path.cwd().joinpath('supporting_files', 'go_propogation_probality_rp_0.3.RData')
result = pyreadr.read_r(str(network_matrix_path)) 
network_matrix = result['pro_p']

network_genes = network_matrix.columns
network_matrix.index = network_genes

network_matrix.shape

(53934, 13)

(19080, 19080)

In [4]:
## clean up unnecessay large variable
del result
gc.collect()

20

### EDA showing concerns with gene info's official_name column

**Optional code chunks (don't have to run).** 

First, rows may have the same official_name values but different information (Name, alias_symbol, genomic coordinates). All rows have unique Name column values (Ensembl IDs), but there are some rows with shared official_name values and some rows with shared alias_symbol values.  

I checked rows where official_name is "AGAP4" and where it was "CX3CR1". I found that **rows with the same official_name actually describe different genes. The IDs in the alias_symbol and Name columns are correct and reliable, and the IDs in the official_name column are not reliable.**  

This can be explored in the code chunks below. 

In [5]:
## Look at number of unique identifers in columns with IDs
print('Unique Name values: {}.'.format(len(pd.unique(gene_info.Name))))
print('Unique official_name values: {}.'.format(len(pd.unique(gene_info.official_name))))
print('Unique alias_symbol values: {}.'.format(len(pd.unique(gene_info.alias_symbol))))

Unique Name values: 53934.
Unique official_name values: 25752.
Unique alias_symbol values: 49915.


In [6]:
## find rows of gene information with the same official_name values
ex0 = gene_info[gene_info.duplicated(subset = 'official_name', keep = False)]  
## the duplicated method returns a boolean vector of whether every row contains a duplicated value
## arguments: subset tells what columns to look for duplicate values in. 
##            keep = False parameter will mark all duplicates. 

## restrict view to useful columns: see how other columns' values are diff for same official_name! 
ex0 = ex0.filter(items = ['Name', 'official_name', 'alias_symbol', 'chrom', 'start_hg19', 'end_hg19', 'strand'])

## restrict view to rows where official_name of the gene is in network
## then arrange rows by the official_name values
ex0[ex0['official_name'].isin(network_genes)].sort_values(by = 'official_name')

,Name,official_name,alias_symbol,chrom,start_hg19,end_hg19,strand
17511,ENSG00000188234.8,AGAP4,AGAP4,chr10,46321042,46349323,-
17603,ENSG00000174194.9,AGAP4,AGAP8,chr10,51224677,51252947,-
34240,ENSGR0000197976.6,AKAP17A,AKAP17A,chrY,1660486,1671407,+
32516,ENSG00000197976.6,AKAP17A,AKAP17A,chrX,1710486,1721407,+
17545,ENSG00000186807.9,ANXA8L1,ANXA8L2,chr10,47746936,47763041,+
...,...,...,...,...,...,...,...
25492,ENSG00000258611.1,YBX2,YBX2,chr15,93608818,93609835,-
32519,ENSG00000214717.4,ZBED1,ZBED1,chrX,2404455,2419008,-
34243,ENSGR0000214717.4,ZBED1,ZBED1,chrY,2354455,2369008,-
30399,ENSG00000179909.9,ZNF154,ZNF154,chr19,58208735,58220579,-


Second, if I remove rows where the official_name value is missing (NaN), I lose information on genes in my network.  

In [7]:
## restrict view to rows where...
ex0[ex0['alias_symbol'].isin(network_genes) &    ## alias_symbol of the gene is in network
    ex0['official_name'].isna()                     ## official_name of gene is missing
   ].sort_values('alias_symbol')  ## arrange rows by alias_symbol column values

,Name,official_name,alias_symbol,chrom,start_hg19,end_hg19,strand
44681,ENSG00000158122.6,NaN,AAED1,chr9,99401859,99417599,-
51555,ENSG00000161533.5,NaN,ACOX1,chr17,73937596,73975515,-
45554,ENSG00000151303.10,NaN,AGAP11,chr10,88728247,88784489,+
40596,ENSG00000189045.7,NaN,ANKDD1B,chr5,74907301,74967671,+
42321,ENSG00000157999.5,NaN,ANKRD61,chr7,6071007,6076017,+
...,...,...,...,...,...,...,...
51892,ENSG00000213297.4,NaN,ZNF625,chr19,12242932,12267546,-
43258,ENSG00000215356.3,NaN,ZNF705B,chr8,7783859,7812271,+
42658,ENSG00000234444.3,NaN,ZNF736,chr7,63767837,63810017,+
52087,ENSG00000261221.1,NaN,ZNF865,chr19,56116771,56128635,+


In [8]:
## clean up the dataframe
del ex0
gc.collect()

40

### Must Run: EDA showing unconnected genes in network  

Must run this chunk so the filtering in the next code chunk works. 

There are 1296 genes that have row and column sums of 0.3, with the self-edge (gene A - gene A entry in the matrix) being 0.3. This means that when Gibbs sampling is done, their network factor as a candidate gene is always 0 - leading to a probability weighting of 0 since self-edge weights are not included. The genes are therefore NEVER being picked in a sample. It is then inefficient to keep these genes in the calculation, so I chose to filter them out. 

The code chunk below can be used to explore this. 

In [9]:
## some of the column sums of the matrix are only 0.3 (the probability that the walk stays at the current node), 
##        which is odd. The other columns sum ~ 1. 
## takes me ~10 seconds to run 
networkcolsums = network_matrix.sum(axis = 0)  ## apply to columns
networkcolsums.value_counts()

## some of the columns with this odd column sum
oddcolsums = networkcolsums[networkcolsums == 0.3]
oddcolsums

## looking into column 'AAED1': 
network_matrix['AAED1'].value_counts()  ## there is only 1 non-zero value: 0.3
network_matrix.loc['AAED1','AAED1']  ## 0.3 is the probability of staying in this node

## look at the row sum for this gene: same issue - the values relating this node to other nodes is 0
network_matrix.loc['AAED1',].sum()
## I looked at the row sums for AAR2, ABRACL, ZYG11A: all are 0.3. 

0.300000    1296
1.000114     211
0.999857      77
0.999857      73
0.999857      58
            ... 
0.999943       1
0.999941       1
0.999903       1
0.999931       1
0.999886       1
Length: 15803, dtype: int64

AAED1       0.3
AAR2        0.3
ABRACL      0.3
ACTL10      0.3
AKAP2       0.3
           ... 
YPEL5       0.3
ZBED6CL     0.3
ZC3HAV1L    0.3
ZYG11A      0.3
ZYG11B      0.3
Length: 1296, dtype: float64

0.0    19079
0.3        1
Name: AAED1, dtype: int64

0.3

0.3

### Filtering to get the input gene set (to look for candidates within)


In [10]:
## takes me a few seconds to run to drop the columns/rows

## keep only the rows where the gene symbol ('alias_symbol column) is in my network 
unconnected = oddcolsums.index
## get genes that are connected: removes 1296 genes to leave 17,784
network_matrix.drop(unconnected, axis = 'index', inplace = True)
network_matrix.drop(unconnected, axis = 'columns', inplace = True)
# network_matrix.index = network_matrix.columns  ## in case rownames got removed...
network_genes = network_matrix.columns  ## update
network_matrix.shape

## I want input genes to be in the network (so I have network information)
gene_info = gene_info[gene_info['alias_symbol'].isin(network_genes)]

gene_info.shape 
gene_info['alias_symbol'].nunique()  

(17784, 17784)

(17062, 13)

17017

There are now 17062 rows in gene_info with 17017 unique gene symbols (alias_symbol). The below code chunk shows that there are 90 rows / 2 rows per alias symbol = 45 unique alias symbols. 

In [11]:
## what is going on with these rows with the same alias_symbol value? Looking...
ex1 = gene_info[gene_info.duplicated(subset='alias_symbol', keep=False)].sort_values(by='alias_symbol')
ex1.shape
ex1.groupby('alias_symbol').size() 

(90, 13)

alias_symbol
AKAP17A      2
ASMT         2
ASMTL        2
C17orf49     2
C20orf141    2
CD99         2
CLEC19A      2
CLLU1        2
CORO7        2
CRLF2        2
CSF2RA       2
CYCS         2
DHRSX        2
DIO3         2
DTX2         2
GATC         2
GTPBP6       2
IL3RA        2
IL9R         2
KIAA1456     2
LIMS3        2
MFRP         2
MRPL46       2
MTG1         2
NOL12        2
OR11H12      2
OR1D5        2
OVCA2        2
P2RY8        2
PLCXD1       2
PPP2R3B      2
RNASE12      2
RNASE4       2
RTEL1        2
SAMD4A       2
SEC16B       2
SHOX         2
SLC25A6      2
SPRY3        2
SULT1A3      2
VAMP7        2
YBX2         2
ZBED1        2
ZNF154       2
ZNF625       2
dtype: int64

After [looking up](http://grch37.ensembl.org/Homo_sapiens/Info/Index) through the Name column (ENSEMBL IDs), I found that some 'duplicate' alias symbols were different transcripts - where one was matched the Ensembl version. I went through those manually to create the array of IDs that matched the Ensembl version. 

The rest of the 'duplicate' alias symbols are legit. They are pseudoautosomal regions, which are matched regions on the X and Y chromosomes. These genes therefore have two loci/sets of genomic coordinates for the SAME gene symbol. Source (has more genes than those in my list below): https://www.genenames.org/data/genegroup/#!/group/714

In [12]:
## IDs that match the Ensembl version
correct_versions = np.array([
    'ENSG00000258315.1',  # C17orf49
    'ENSG00000258713.1',  # C20orf141: this row in df has different coordinates from ENSEMBL (ENSEMBL has Chromosome 20: 2,795,633-2,796,479) 
    'ENSG00000261210.1',  # CLEC19A
    'ENSG00000257127.1',  # CLLU1
    'ENSG00000262246.1',  # CORO7
    'ENSG00000172115.4',  # CYCS
    'ENSG00000197406.5',  # DIO3
    'ENSG00000091073.13', # DTX2
    'ENSG00000257218.1',  # GATC
    'ENSG00000250305.3',  # KIAA1456
    'ENSG00000256977.2',  # LIMS3: this row in df has different coordinates from ENSEMBL (ENSEMBL has Chromosome 2: 110,656,005-110,677,180) 
    'ENSG00000235718.3',  # MFRP: this alias has two legit-looking ENSG IDs, I picked the one with more transcripts/orthologs
    'ENSG00000259494.1',  # MRPL46
    'ENSG00000148824.12', # MTG1
    'ENSG00000100101.12', # NOL12: this row in df has different coordinates from ENSEMBL (ENSEMBL has Chromosome 22: 38,077,680-38,170,137) 
    'ENSG00000257115.1',  # OR11H12
    'ENSG00000262628.1',  # OR1D5
    'ENSG00000262664.1',  # OVCA2
    'ENSG00000258436.1',  # RNASE12
    'ENSG00000258818.1',  # RNASE4
    'ENSG00000258366.2',  # RTEL1
    'ENSG00000020577.9',  # SAMD4A
    'ENSG00000120341.10', # SEC16B: this row in df has different coordinates from ENSEMBL (ENSEMBL has Chromosome 1: 177,893,091-177,953,438) 
    'ENSG00000261052.1',  # SULT1A3 
    'ENSG00000006047.8',  # YBX2
    'ENSG00000179909.9',  # ZNF154
    'ENSG00000257591.1'   # ZNF625
], dtype=object)

## Pseudoautosomal genes
pseudoautosomal_list = np.array(['AKAP17A', 'ASMT', 'ASMTL', 'CD99', 'CRLF2', 'CSF2RA', 'DHRSX', 'GTPBP6',
                                 'IL3RA', 'IL9R', 'P2RY8', 'PLCXD1', 'PPP2R3B', 'SHOX', 'SLC25A6', 'SPRY3', 
                                 'VAMP7', 'ZBED1'], dtype=object)  

## for dataframe isin() method: 
## dict key is column name, value is list of values to check for in that column 
criteria = {'Name': correct_versions, 'alias_symbol': pseudoautosomal_list}
## saves rows we don't want to keep: it's NOT the correct Ensembl ID OR it's NOT a pseudoautosomal gene
ex1 = ex1[~ ex1.isin(criteria).any('columns')]  
## reference: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html?highlight=isin#indexing-with-isin

## get dataframe with rows from ex1 removed
gene_info = gene_info[~ gene_info['Name'].isin(ex1['Name'])]

## stats for filtered dataframe of gene info  
gene_info.shape
gene_info['alias_symbol'].nunique()

(17035, 13)

17017

The final gene_info dataframe only has genes in the connected part of the gene-gene network. There are 17035 rows (unique Ensembl IDs in the 'Name' column) and 17017 unique gene symbols (alias_symbol column). The duplicate gene symbols are from 18 pseudoautosomal genes.    

In [13]:
## clean up the dataframe
del ex1, criteria, correct_versions, unconnected, networkcolsums, oddcolsums
gc.collect()

20

### Get GWAS SNP info
**CHANGES from original code:** removed check on flank_length > 0  

**Notes on parameters:**
* SNP_file_path: path of file with GWAS SNPs. The file must include three columns named (case matters): SNP, Chr, and Pos_hg19  
* flank: integer (in bp, recommended by iRIGS authors: 1000000 for a 2MB flanking region total). The flanking region is the genomic region between *flank* base-pairs upstream and *flank* base-pairs downstream of a GWAS SNP. In other words, *flank* is 1/2 the length of the flanking region. Genes in the flanking region will be used as candidate genes during Gibbs sampling    

In [14]:
## Parameters
SNP_file_path = Path.cwd().joinpath('SNP_file', 'SCZ_108_loci')
flank = 1000000

In [15]:
## There was a check here to halt the program if the variable flank < 0: I removed it

## read GWAS SNP file: this will go forward w/o error even if the columns specified in dtype aren't there
df_gwas_snps = pd.read_csv(SNP_file_path, sep='\t', header=0, \
                           dtype={'SNP': object, 'Chr': object, 'Pos_hg19': 'Int64'})

## check if the required columns are in the table
if not {'SNP', 'Chr', 'Pos_hg19'}.issubset(df_gwas_snps.columns):
    raise KeyError(\
        'At least one of the following required columns is missing from the GWAS SNP file: SNP, Chr, Pos_hg19')

## if needed, in Chr column, add "chr" prefix to each chromosome name 
## grabs first entry of Chr column and checks that it starts with chr
if not df_gwas_snps['Chr'][1].startswith('chr'):  
    df_gwas_snps['Chr'] = 'chr' + df_gwas_snps['Chr']

## look at dataframe to check that it was imported/processed correctly
df_gwas_snps.head()
df_gwas_snps.shape

,SNP,Chr,Pos_hg19
0,rs115329265,chr6,28712247
1,rs1702294,chr1,98501984
2,rs11191419,chr10,104612335
3,rs2007044,chr12,2344960
4,rs4129585,chr8,143312933


(108, 3)

### Find candidate genes around GWAS SNPs      

Because pseudoautosomal genes are on different chromosomes, this method of getting candidate genes that neighbor SNPs will NEVER pick up both copies of a pseudoautosomal gene. **This means that within each candidate gene set for a SNP, alias_symbol values will be unique and we can use this column going forward to ID genes/rows.** 

Note that this version (vectorized, running on numpy arrays) runs about 5X faster than other methods (stored in the commented code chunk). 

In [16]:
# %%timeit  ## for timing

candidate_genes = pd.merge(df_gwas_snps, gene_info, left_on = 'Chr', right_on = 'chrom', how="left")
## creates dataframe where genes are appended to SNPs if chromosome number matches

def snp_to_candidates(SNP_name, SNP_pos, gene_start, gene_end, flank_length):
    """
    Arguments: 
    all numpy arrays must be the same length, ideally converted from columns of the same pandas DataFrame
    
    SNP_name: numpy array of strings, SNP names
    SNP_pos: numpy array of int, SNP position in hg19
    gene_start: numpy array of int, gene start position in hg19
    gene_end: numpy array of int, gene end position in hg19
    flank_length: int, from original arguments 
                  range on one side of SNP to look for candidate genes
    
    Returns: pandas Boolean array (same length as input arrays) for whether genes are within flanking region
             and are therefore candidate genes 
    """
    ## to add/subtract flank from SNP position array, need this in array form
    flank_vector = np.repeat(flank_length, len(SNP_pos))
    
    ## get arrays with flank start and end for SNP in each row
    flank_start = np.maximum(SNP_pos - flank_vector, 0)  ## beginning of chromosome or upstream end
    flank_end = SNP_pos + flank_vector 
    ## downstream end: doesn't cause error if this is beyond length of chromosome
    
    ## boolean vector: True if gene starts after flank_start and before flank_end.
    criteria_start = (gene_start > flank_start) & (gene_start < flank_end)
    ## boolean vector: True if gene ends after flank_start and before flank_end.
    criteria_end = (gene_end > flank_start) & (gene_end < flank_end)
    ## logical OR. This means that genes that partially overlap the flanking region 
    ## (end in flanking region or begin in flanking region) are also included
    index_logic = criteria_start | criteria_end
    return index_logic

# # used to measure performance of function: notice the function only runs once
# %lprun -f snp_to_candidates temptry_logic = snp_to_candidates(candidate_genes['SNP'].values, \
#                                                               candidate_genes['Pos_hg19'].values, \
#                                                               candidate_genes['start_hg19'].values,\
#                                                               candidate_genes['end_hg19'].values, flank)

candidate_logic = snp_to_candidates(candidate_genes['SNP'].values, \
                                    candidate_genes['Pos_hg19'].values, \
                                    candidate_genes['start_hg19'].values,\
                                    candidate_genes['end_hg19'].values, 
                                    flank)

candidate_genes = candidate_genes[candidate_logic]
candidate_genes.rename(columns = {'Chr':'SNP_chr', 'Pos_hg19':'SNP_pos_hg19'}, inplace = True)
## chunk timing
# 85.6 ms ± 704 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)

Below is other attempts in "vectorizing". It gets the same results as the above code. However, it is slower. Uncomment to run and see timing. 

In [17]:
# %%timeit  

# def snp_to_candidates_slower(SNP_name, chr_num, pos_hg19, flank_length, gene_df):
#     """
#     Arguments:
#     SNP_name: string, name of the SNP
#     chr_num: string, format: "chr1"
#     pos_hg19: int, position in hg19 
#     flank_length: int, from original arguments 
#                   range on one side of SNP to look for candidate genes
#     gene_df: dataframe with gene info. MUST HAVE chrom (string, format "chr1"), 
#              start_hg19 (int), end_hg19 (int) columns (position info). 
#     """
#     flank_start = max(pos_hg19 - flank_length, 0)  ## beginning of chromosome or upstream end
#     flank_end = pos_hg19 + flank_length 
#     ## downstream end: doesn't cause error if this is beyond length of chromosome
    
#     ## subset to find genes in same chromosome as SNP
#     temp = gene_df[gene_df['chrom'] == chr_num]  
#     ## boolean vector: True if gene starts after flank_start and before flank_end.
#     criteria_start = (temp['start_hg19'] > flank_start) & (temp['start_hg19'] < flank_end)
#     ## boolean vector: True if gene ends after flank_start and before flank_end.
#     criteria_end = (temp['end_hg19'] > flank_start) & (temp['end_hg19'] < flank_end)
#     ## logical OR. This means that genes that partially overlap the flanking region 
#     ## (end in flanking region or begin in flanking region) are also included
#     temp = temp[criteria_start | criteria_end]
    
#     if len(temp) > 0:  ## if candidate genes were found
#         ## append matched GWAS SNP info to each row (gene)
#         temp['SNP'] = SNP_name
#         temp['SNP_chr'] = chr_num
#         temp['SNP_pos_hg19'] = pos_hg19
        
#         ## get a list of rows (each row is a list)
#         temp_list = temp.values.tolist()  ## tolist is a numpy function
#         return temp_list
#     else:
#         return []  ## return empty list

# candidate_genes_list = []

# ## apply function over rows
# # templist = df_gwas_snps.apply(lambda row: snp_to_candidates_slower(row['SNP'], row['Chr'], \
# #                                                             row['Pos_hg19'], flank, df_gene_info), axis=1)
# # for alist in templist:
# #     candidate_genes_list.extend(alist)
# ## speed: 496 ms ± 10.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

# ## iterate over rows: =
# for row in df_gwas_snps.itertuples(index=False):
#     temp = snp_to_candidates_slower(row.SNP, row.Chr, row.Pos_hg19, flank, df_gene_info)
#     candidate_genes_list.extend(temp)
# ## speed: 476 ms ± 7.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

# candidate_genes_slower = pd.DataFrame(candidate_genes_list, columns = ['Name', 'Description', 'chrom', 'type', 
#                                                                 'start_hg19', 'end_hg19','strand', 'ts_id', 
#                                                                 'gene_type', 'gene_status', 'loci_level',
#                                                                 'alias_symbol', 'official_name', 'SNP', 
#                                                                'SNP_chr', 'SNP_pos_hg19'])

In [18]:
## look at candidate gene dataframe
candidate_genes.shape
candidate_genes.head()

## save numpy arrays with SNPs and genes (no duplicates)
SNP_loci = candidate_genes['SNP'].unique()
candidate_genelist = candidate_genes['alias_symbol'].unique()

print('{genes} unique ENSG IDs (genes) near {loci} GWAS SNPs were found...\n'.format(
      genes = len(candidate_genelist), loci = len(SNP_loci)))

(1637, 16)

,SNP,SNP_chr,SNP_pos_hg19,Name,Description,chrom,type,start_hg19,end_hg19,strand,ts_id,gene_type,gene_status,loci_level,alias_symbol,official_name
133,rs115329265,chr6,28712247,ENSG00000185130.4,HIST1H2BL,chr6,gene,27775257,27775709,-,ENSG00000185130.4,protein_coding,KNOWN,2,HIST1H2BL,HIST1H2BL
134,rs115329265,chr6,28712247,ENSG00000182611.3,HIST1H2AJ,chr6,gene,27782112,27782607,-,ENSG00000182611.3,protein_coding,KNOWN,2,HIST1H2AJ,HIST1H2AJ
135,rs115329265,chr6,28712247,ENSG00000196374.5,HIST1H2BM,chr6,gene,27782822,27783267,+,ENSG00000196374.5,protein_coding,KNOWN,2,HIST1H2BM,HIST1H2BM
136,rs115329265,chr6,28712247,ENSG00000233822.2,HIST1H2BN,chr6,gene,27806323,27819974,+,ENSG00000233822.2,protein_coding,KNOWN,2,HIST1H2BN,HIST1H2BN
137,rs115329265,chr6,28712247,ENSG00000184357.3,HIST1H1B,chr6,gene,27834570,27835359,-,ENSG00000184357.3,protein_coding,KNOWN,2,HIST1H1B,HIST1H1B


1577 unique ENSG IDs (genes) near 107 GWAS SNPs were found...



There are 1637 rows in the candidate_genes dataframe but only 1577 unique genes, which means that **some SNP loci have the same candidate genes**. As shown in the code chunk below, there are no pseudoautosomal genes in this set. Also, one SNP had no candidate genes and is not in the dataframe. 

In [19]:
# EDA: 
## There are no pseudoautosomal genes in this set.   
candidate_genes[candidate_genes['alias_symbol'].isin(pseudoautosomal_list)].empty
## trying to get a subset of pseudoautosomal genes -> get an empty dataframe 

## one SNP had no candidate genes found: chr2_146436222_I
candidate_genes.query("SNP == 'chr2_146436222_I'").empty
## trying to get a subset for this SNP -> get an empty dataframe 

True

True

In [20]:
## clean up 
del candidate_logic
gc.collect()

20

## Processing network and supporting evidence

### Filter network adjacency matrix

In [21]:
## slice the matrix to only include candidate genes
network_matrix = network_matrix.loc[candidate_genelist, candidate_genelist]  ## end up (1577, 1577)
network_matrix.shape

## to match ENSG IDs in supporting evidence files, take version number off ENSG IDs 
candidate_genes['Name'] = candidate_genes['Name'].str.slice(0, 15)

(1577, 1577)

### Supporting evidence 1: distance from SNP to gene transcription start site (TSS) (DTS)

In [22]:
def get_DTS(SNP_pos, gene_strand, gene_start, gene_end):
    """
    Arguments: 
    all arrays must be the same length, ideally columns from the same pandas DataFrame
    
    SNP_pos: numpy array of int, SNP position in hg19
    gene_strand: numpy array of strings, what strand the gene is on
    gene_start: numpy array of int, gene start position in hg19
    gene_end: numpy array of int, gene end position in hg19
    
    Returns: an numpy array of int, linear sequence distance from SNP
    """
    ## want to iterate over indices for these arrays
    array_length = len(SNP_pos)

    ## list to build 
    DTS = []
    
    for idx in range(array_length):
        if gene_strand[idx] == '+':  # if gene is on strand + 
            temp = abs(gene_start[idx] - SNP_pos[idx])  # TSS is at gene start
        else:
            temp = abs(gene_end[idx] - SNP_pos[idx])  # TSS is at gene end
        DTS.append(temp)
    
    DTS = np.array(DTS)
    return DTS

## used to measure performance of function
candidate_genes['DTS'] = get_DTS(candidate_genes['SNP_pos_hg19'].values, \
                                          candidate_genes['strand'].values, \
                                          candidate_genes['start_hg19'].values,\
                                          candidate_genes['end_hg19'].values) 

In [23]:
candidate_genes.head()

,SNP,SNP_chr,SNP_pos_hg19,Name,Description,chrom,type,start_hg19,end_hg19,strand,ts_id,gene_type,gene_status,loci_level,alias_symbol,official_name,DTS
133,rs115329265,chr6,28712247,ENSG00000185130,HIST1H2BL,chr6,gene,27775257,27775709,-,ENSG00000185130.4,protein_coding,KNOWN,2,HIST1H2BL,HIST1H2BL,936538
134,rs115329265,chr6,28712247,ENSG00000182611,HIST1H2AJ,chr6,gene,27782112,27782607,-,ENSG00000182611.3,protein_coding,KNOWN,2,HIST1H2AJ,HIST1H2AJ,929640
135,rs115329265,chr6,28712247,ENSG00000196374,HIST1H2BM,chr6,gene,27782822,27783267,+,ENSG00000196374.5,protein_coding,KNOWN,2,HIST1H2BM,HIST1H2BM,929425
136,rs115329265,chr6,28712247,ENSG00000233822,HIST1H2BN,chr6,gene,27806323,27819974,+,ENSG00000233822.2,protein_coding,KNOWN,2,HIST1H2BN,HIST1H2BN,905924
137,rs115329265,chr6,28712247,ENSG00000184357,HIST1H1B,chr6,gene,27834570,27835359,-,ENSG00000184357.3,protein_coding,KNOWN,2,HIST1H1B,HIST1H1B,876888


### Supporting evidence 2: Schizophrenia (SCZ) de novo mutation (DNM) data  
Thoughts: Why did the authors remove "start_lost" information (one row of info)?  

In [24]:
## get DNM data from one file
dnm_path_s3 = Path.cwd().joinpath('supporting_files', 'SCZ_DNM', \
                                  '2014_Feb_De_novo_mutation_in_SCZ_Nature_623_trios_s3.txt')
dnm = pd.read_table(dnm_path_s3)
## note that the child phenoypes are ASD, control, ID, SZ, Unaffected sibling
dnm = dnm[dnm['Child phenotype'] == 'SZ']  ## get DNM in individuals with schizophrenia  

## get DNM from the other file
dnm_path_s2 = Path.cwd().joinpath('supporting_files', 'SCZ_DNM', \
                                  '2014_Feb_De_novo_mutation_in_SCZ_Nature_623_trios_s2.txt')
fromer = pd.read_table(dnm_path_s2)
fromer['Child phenotype'] = np.repeat('SZ', len(fromer))
fromer['Study'] = np.repeat('Fromer', len(fromer))
## all of these are DNM in individuals with schizophrenia   

dnm = dnm.merge(fromer, how = 'outer', on = ['Child phenotype', 'Study', 'Locus', 'Reference allele', \
                 'Alternate allele', 'Gene annotations', 'Genes', 'Transcripts', 'Transcript annotations', \
                 'Codon changes', 'Protein changes', 'Tri-nucleotide base context (forward strand) for SNV'])

## There are 10 types of gene annotations. Authors kept 6: "esplice", "frameshift", "nonsense", "missense", "codon-deletion", "code-insertion". More efficient to take out the others
removed_annots = ['silent', 'silent,missense', 'silent,silent', 'start-lost']
dnm = dnm[~ dnm['Gene annotations'].isin(removed_annots)]

In [25]:
dnm.head()

,Child phenotype,Study,Locus,Reference allele,Alternate allele,Gene annotations,Genes,Transcripts,Transcript annotations,Codon changes,...,Mother genotype meta-information,Sequencing center,Inferred parental origin of mutation,Proband ID,Proband gender,Family history,Paternal age at proband birth,Maternal age at proband birth,Proband age at onset of disease,"Does the proband have a de novo CNV (Kirov, et al., 2012)?"
0,SZ,Girard,chr19:36673714,T,C,missense,ZNF565,"NM_001042474,NM_152477","missense,missense","c.1154CAT>CGT,c.1154CAT>CGT",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,SZ,Girard,chr21:16338349,T,G,missense,NRIP1,NM_003489,missense,c.2165AAA>ACA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,SZ,Girard,chr12:57579450,C,A,nonsense,LRP1,NM_002332,nonsense,c.6600TAC>TAA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,SZ,Girard,chr17:79637360,A,G,missense,CCDC137,NM_199287,missense,c.374TAT>TGT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,SZ,Girard,chr3:122146472,C,A,nonsense,KPNA1,"NM_002264,NR_026698","nonsense,npcRNA","c.1342GAA>TAA,.",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
## clean up 
del fromer, dnm_path_s2, dnm_path_s3, removed_annots
gc.collect()

51

### Supporting evidence 3: SCZ-specific differential expression (DE) data  
This data is already in p-value form  

In [27]:
de_path = Path.cwd().joinpath('supporting_files', 'SCZ_DE', \
          'CMC_MSSM-Penn-Pitt_DLPFC_mRNA_IlluminaHiSeq2500_gene-adjustedSVA-differentialExpression-includeAncestry-DxSCZ-DE.tsv')
dexpr = pd.read_table(de_path)

dexpr.head()

,genes,MAPPED_genes,logFC,CI.L,CI.R,AveExpr,t,P.Value,adj.P.Val,B
0,ENSG00000141433,ADCYAP1,0.35520,0.27200,0.43840,4.472,8.386,4.159000e-16,6.830000e-12,25.90230
1,ENSG00000138640,FAM13A,-0.09541,-0.12470,-0.06606,6.365,-6.387,3.589000e-10,2.908000e-06,12.69002
2,ENSG00000135299,ANKRD6,0.13390,0.09229,0.17550,5.291,6.322,5.312000e-10,2.908000e-06,12.35552
3,ENSG00000104327,CALB1,0.19220,0.13140,0.25290,5.241,6.217,9.951000e-10,4.085000e-06,11.75607
4,ENSG00000125170,DOK4,0.10560,0.07147,0.13970,3.795,6.079,2.254000e-09,7.405000e-06,11.01800


In [28]:
## clean up 
del de_path
gc.collect()

40

### Supporting evidence 4-5: brain Hi-C data (CP + GZ)

In [29]:
cp_path = Path.cwd().joinpath('supporting_files', 'BrainHiC', 'S22_TSS_CP.txt')
gz_path = Path.cwd().joinpath('supporting_files', 'BrainHiC', 'S23_TSS_GZ.txt')

cp = pd.read_table(cp_path)  ## cp has 221069 rows
cp.drop(labels = 'Unnamed: 7', axis = 'columns', inplace=True)
gz = pd.read_table(gz_path)  ## gz has 228323 rows

## make a column of enhancer coordinate info that makes it easy to compare positions  
cp['cp_enhancer_no'] = cp['chr'].str.cat(cp[['interacting_bin_start', 'interacting_bin_end']].astype(str), \
                  sep = ':')
gz['gz_enhancer_no'] = gz['chr'].str.cat(gz[['interacting_bin_start', 'interacting_bin_end']].astype(str), \
                  sep = ':')

## count how many unique enhancers (by chr, coordinates) per gene in dataset
cp_counts = cp[['ENSGID_for_TSS', 'cp_enhancer_no']].groupby('ENSGID_for_TSS')['cp_enhancer_no'].nunique().to_frame()
cp_counts.reset_index(inplace = True)  ## 34255 ENSG IDs/rows

gz_counts = gz[['ENSGID_for_TSS', 'gz_enhancer_no']].groupby('ENSGID_for_TSS')['gz_enhancer_no'].nunique()
  ## 34255 ENSG IDs/rows
gz_counts = gz_counts.to_frame()
gz_counts.reset_index(inplace = True)  ## 35019 ENSG IDs/rows

## adds columns to candidate_gene df 
candidate_genes = candidate_genes.merge(cp_counts, how = 'left', left_on = 'Name', right_on = 'ENSGID_for_TSS')
candidate_genes = candidate_genes.merge(gz_counts, how = 'left', left_on = 'Name', right_on = 'ENSGID_for_TSS')
candidate_genes.drop(labels = ['ENSGID_for_TSS_x', 'ENSGID_for_TSS_y'], axis = 'columns', inplace=True)

## replace NAs with 0 for cp_enhancer_no, gz_enhancer_no
candidate_genes['cp_enhancer_no'].fillna(0, downcast='infer', inplace=True)
candidate_genes['gz_enhancer_no'].fillna(0, downcast='infer', inplace=True)

candidate_genes.head()

,SNP,SNP_chr,SNP_pos_hg19,Name,Description,chrom,type,start_hg19,end_hg19,strand,ts_id,gene_type,gene_status,loci_level,alias_symbol,official_name,DTS,cp_enhancer_no,gz_enhancer_no
0,rs115329265,chr6,28712247,ENSG00000185130,HIST1H2BL,chr6,gene,27775257,27775709,-,ENSG00000185130.4,protein_coding,KNOWN,2,HIST1H2BL,HIST1H2BL,936538,10,10
1,rs115329265,chr6,28712247,ENSG00000182611,HIST1H2AJ,chr6,gene,27782112,27782607,-,ENSG00000182611.3,protein_coding,KNOWN,2,HIST1H2AJ,HIST1H2AJ,929640,8,9
2,rs115329265,chr6,28712247,ENSG00000196374,HIST1H2BM,chr6,gene,27782822,27783267,+,ENSG00000196374.5,protein_coding,KNOWN,2,HIST1H2BM,HIST1H2BM,929425,8,9
3,rs115329265,chr6,28712247,ENSG00000233822,HIST1H2BN,chr6,gene,27806323,27819974,+,ENSG00000233822.2,protein_coding,KNOWN,2,HIST1H2BN,HIST1H2BN,905924,6,7
4,rs115329265,chr6,28712247,ENSG00000184357,HIST1H1B,chr6,gene,27834570,27835359,-,ENSG00000184357.3,protein_coding,KNOWN,2,HIST1H1B,HIST1H1B,876888,4,5


In [30]:
## clean up 
del cp_counts, gz_counts
gc.collect()

44

### Supporting evidence 6: capture Hi-C data (cell line, not specific to brain)  

In [31]:
capHiC_path = Path.cwd().joinpath('supporting_files', 'capHiC', 'GM12878_DRE_number')
capHiC = pd.read_table(capHiC_path)

candidate_genes = candidate_genes.merge(capHiC[['Name', 'cap4_enhancer_no']], how = 'left', \
                      on = 'Name')

candidate_genes.head()

,SNP,SNP_chr,SNP_pos_hg19,Name,Description,chrom,type,start_hg19,end_hg19,strand,ts_id,gene_type,gene_status,loci_level,alias_symbol,official_name,DTS,cp_enhancer_no,gz_enhancer_no,cap4_enhancer_no
0,rs115329265,chr6,28712247,ENSG00000185130,HIST1H2BL,chr6,gene,27775257,27775709,-,ENSG00000185130.4,protein_coding,KNOWN,2,HIST1H2BL,HIST1H2BL,936538,10,10,223.0
1,rs115329265,chr6,28712247,ENSG00000182611,HIST1H2AJ,chr6,gene,27782112,27782607,-,ENSG00000182611.3,protein_coding,KNOWN,2,HIST1H2AJ,HIST1H2AJ,929640,8,9,0.0
2,rs115329265,chr6,28712247,ENSG00000196374,HIST1H2BM,chr6,gene,27782822,27783267,+,ENSG00000196374.5,protein_coding,KNOWN,2,HIST1H2BM,HIST1H2BM,929425,8,9,211.0
3,rs115329265,chr6,28712247,ENSG00000233822,HIST1H2BN,chr6,gene,27806323,27819974,+,ENSG00000233822.2,protein_coding,KNOWN,2,HIST1H2BN,HIST1H2BN,905924,6,7,26.0
4,rs115329265,chr6,28712247,ENSG00000184357,HIST1H1B,chr6,gene,27834570,27835359,-,ENSG00000184357.3,protein_coding,KNOWN,2,HIST1H1B,HIST1H1B,876888,4,5,130.0


### Supporting evidence 7: FANTOM5 data 
I do get 66899 enhancer-promoter links at end of filtering FDR < 1 (mentioned in original paper)!    

In [32]:
FANTOM_path = Path.cwd().joinpath('supporting_files', 'Fantom5', 'enhancer_tss_associations.bed.txt')
FANTOM5_data = pd.read_table(FANTOM_path)  ## 66942 rows
 
## parsing name column: adds NA to 35 rows
tempFANTOM = FANTOM5_data['name'].str.split(pat = ';|:', expand = True)  ## splits name column into many columns
tempFANTOM.drop(labels = [4, 6], axis = 'columns', inplace=True)  ## drop Rstat/FDR strings
tempFANTOM.columns = ['chr', 'pos_range', 'NCBI_ID', 'gene', 'Rstat', 'FDR']  ## assign columns
FANTOM5_data = tempFANTOM.join(FANTOM5_data)

## removes those 35 rows. have 66907 left
FANTOM5_data.dropna(axis = "index", inplace = True)

## select all genes with FDR < 1 
FANTOM5_data['FDR'] = FANTOM5_data['FDR'].astype(np.float64)
FANTOM5_data.query('FDR < 1', inplace = True)
## removes 8 rows: left with 66899 rows

## count how many unique enhancers (identified by position) per gene in dataset
FANTOM5_enhancer_count = FANTOM5_data[['gene', 'pos_range']].rename(columns = {'pos_range':'fantom5_enhancer_no'})
FANTOM5_enhancer_count = FANTOM5_enhancer_count.groupby('gene')['fantom5_enhancer_no'].nunique().to_frame()
FANTOM5_enhancer_count.reset_index(inplace = True) 
## 11602 rows (unique gene symbols)

## adds fantom5_enhancer_no column to table
candidate_genes = candidate_genes.merge(FANTOM5_enhancer_count, how = 'left', left_on = 'alias_symbol', \
                                        right_on = 'gene')
candidate_genes.drop(labels = 'gene', axis = 'columns', inplace=True)
# candidate_genes[candidate_genes['fantom5_enhancer_no'].isna()].shape
## 597 NAs were added so need to replace those with 0. 
candidate_genes['fantom5_enhancer_no'].fillna(0, downcast='infer', inplace=True)

candidate_genes.head()

,SNP,SNP_chr,SNP_pos_hg19,Name,Description,chrom,type,start_hg19,end_hg19,strand,...,gene_type,gene_status,loci_level,alias_symbol,official_name,DTS,cp_enhancer_no,gz_enhancer_no,cap4_enhancer_no,fantom5_enhancer_no
0,rs115329265,chr6,28712247,ENSG00000185130,HIST1H2BL,chr6,gene,27775257,27775709,-,...,protein_coding,KNOWN,2,HIST1H2BL,HIST1H2BL,936538,10,10,223.0,4
1,rs115329265,chr6,28712247,ENSG00000182611,HIST1H2AJ,chr6,gene,27782112,27782607,-,...,protein_coding,KNOWN,2,HIST1H2AJ,HIST1H2AJ,929640,8,9,0.0,4
2,rs115329265,chr6,28712247,ENSG00000196374,HIST1H2BM,chr6,gene,27782822,27783267,+,...,protein_coding,KNOWN,2,HIST1H2BM,HIST1H2BM,929425,8,9,211.0,4
3,rs115329265,chr6,28712247,ENSG00000233822,HIST1H2BN,chr6,gene,27806323,27819974,+,...,protein_coding,KNOWN,2,HIST1H2BN,HIST1H2BN,905924,6,7,26.0,6
4,rs115329265,chr6,28712247,ENSG00000184357,HIST1H1B,chr6,gene,27834570,27835359,-,...,protein_coding,KNOWN,2,HIST1H1B,HIST1H1B,876888,4,5,130.0,4


In [33]:
## clean up 
del FANTOM_path, FANTOM5_enhancer_count, capHiC_path
gc.collect()

35

## Calculating and combining p-values

**CHANGES from original code:**
* replaced missing values in capHiC column with 0, to match how missing values were handled in other enhancer columns 
* kept surrogate Bayes factor for each gene-SNP pair (use SNP ID and alias_symbol to retrieve during Gibbs sampling)  

Questions (also mentioned in Intro):  
{? why did original code appear to keep only one "extra_weight" (surrogate Bayes factor, the first match) per unique gene/"official_name" value?}  
{? The Mahalanobis transformation is supposed to create variables that follow the univariable Gaussian (normal) distribution but I don't think they do based on the EDA...}  
{? Why is dnm probability calculated using a hard-coded number: 20,000?}  
{? why get negative log of the final "extra_weights"? Maybe to get a continuous measure (https://lesslikely.com/statistics/s-values/), or to get bigger numbers when meta p-value is small?}  

### Mahalanobis transformation (ZCA whitening)   

* Done on DTS, enhancer columns of supporting data (capHiC, FANTOM5, brain-specific HiC)  
* This creates uncorrelated variables, which can then be used to find p-values 

In [34]:
## process data to get mean-centered values  
zca_cols = ['DTS', 'cap4_enhancer_no', 'fantom5_enhancer_no', 'cp_enhancer_no', 'gz_enhancer_no']

extra_evidence = candidate_genes[zca_cols].copy()
extra_evidence.index = candidate_genes['alias_symbol']  ## trying this to keep track of gene names

## show rows with missing values
extra_evidence[extra_evidence.isnull().any(axis=1)]

extra_evidence['cap4_enhancer_no'].fillna(0, inplace = True)

,DTS,cap4_enhancer_no,fantom5_enhancer_no,cp_enhancer_no,gz_enhancer_no
alias_symbol,,,,,
C10orf95,396281,NaN,0,2,2
RIPPLY2,282711,NaN,1,0,1
C1orf233,851625,NaN,0,5,6


Notice that only one column has NA values: cap4_enhancer_no. This is because DTS is calculated from genomic coordinates and the other enhancer columns replaced their missing values with 0 (in the original R code, the default value for the columns is 0: see extra_evi.R).  

The original R code replaces the missing values here, aka cap4_enhancer_no's missing values, with the median for the column. I want to treat this column the same as the other enhancer columns, so I replace the 3 missing values with 0. 

Next is calculating the Mahalanobis transformation.  

In [35]:
## first calculate covariance matrix, and SVD on the covariance matrix
cov_extra = extra_evidence.cov().values  ## gives 5 x 5 (pairwise covariances) numpy ndarray

svd_u, svd_diag, svd_vh = np.linalg.svd(cov_extra) 
## u and v are the same 5 x 5 matrix, diag = singular values

## find 1 / sqrt () for each value in d (same as () ^ (-0.5)). Then make these values 
##      into a diagonal matrix = new_s_diag (values only on diagonal, 0 otherwise). 
new_s_diag = np.diag(v = svd_diag**(-0.5))
## make new set of transformations: U * new_s_diag * V(transpose). 
##      Looks like "making a new covariance matrix for a new set of variables". 
svd_newtrans = svd_u @ new_s_diag @ svd_vh
## @ is matrix multiplication, on 5 x 5 matrices

The Mahalanobis transformation is done on mean-centered values.  

In [36]:
## get mean-centered values 
extra_evidence = extra_evidence.apply(lambda x: x - x.mean(), axis = 0)  ## applies to each column

## transforming mean-centered values  
extra_evidence = (svd_newtrans @ extra_evidence.T).T
## 5 x 5 matrix * 5 x 1732 matrix = 5 x 1732 matrix. 
## Then transpose to get it back in the same form as original df: 1732 genes x 5 variables. 

## the columns are "decorrelated versions" of the original variables
##  give them the same column names
extra_evidence.columns = zca_cols

In [37]:
extra_evidence.head()

,DTS,cap4_enhancer_no,fantom5_enhancer_no,cp_enhancer_no,gz_enhancer_no
alias_symbol,,,,,
HIST1H2BL,1.423880,2.489901,-0.069589,0.579968,0.453113
HIST1H2AJ,1.401154,-1.219034,0.184902,0.183018,0.585704
HIST1H2BM,1.400452,2.290412,-0.048384,0.165730,0.551136
HIST1H2BN,1.323046,-0.789068,0.497234,-0.039871,0.365755
HIST1H1B,1.227420,0.942483,0.063548,-0.258647,0.140395


With the current setup, we would need to calculate the p-value for the lower tail for the DTS variable and the p-value for the upper tail for the other variables. This is because...  
* DTS: the smaller the number (distance to SNP < expected), the more likely it is to be a disease gene (higher weight)  
* for the others: the larger the number (number of DRE-promoter links > expected), the more likely it is to be a disease gene (higher weight). Recall the assumption made (Description of iRIGS above).    

For the other columns, we flip their signs to flip the tails. Now for all variables, the smaller the number, the more likely it is to be a disease gene.  

Note: the Mahalanobis transformation is supposed to create variables that follow the univariable Gaussian (normal) distribution but I feel like it doesn't...see the last code chunk for this section

In [38]:
larger_likely_disease = ['cap4_enhancer_no', 'fantom5_enhancer_no', 'cp_enhancer_no', 'gz_enhancer_no']

extra_evidence = extra_evidence.apply(lambda x: -x if x.name in larger_likely_disease else x)

In [39]:
## notice how small the p-values for the normality tests are 
## ref: https://docs.scipy.org/doc/scipy/reference/tutorial/stats.html#special-tests-for-normal-distributions
stats.normaltest(extra_evidence)

NormaltestResult(statistic=array([ 399.3259364 ,  128.09458355, 1887.39056917,  818.13328282,
        650.04895805]), pvalue=array([1.93853957e-087, 1.52972936e-028, 0.00000000e+000, 2.21113319e-178,
       6.97689643e-142]))

### Make p-values from transformed variables  

In [40]:
## ref for stats.norm.cdf being the same as R's pnorm : https://stackoverflow.com/a/42065440
extra_evidence = extra_evidence.apply(lambda x: stats.norm.cdf(x))

In [41]:
del larger_likely_disease, zca_cols, cov_extra, svd_u, svd_diag, svd_vh, new_s_diag, svd_newtrans
gc.collect()

100

### Add p-values for DNM and DE data 

Add and process p-values from DNM (de-novo mutation) and DE (differential expression) data. 

Notice:
* This data is schizophrenia specific.  
* The hard-coding denominator of 20,000 for the DNM probability calculation. Perhaps can be replaced with a different number?

In [42]:
## DNM data 
dnm_prob = dnm['Genes'].nunique() / 20000

## only 60 candidate genes have schizophrenia-associated DNM: they are assigned the dnm_prob
# len(extra_evidence[extra_evidence.index.isin(dnm['Genes'])])
extra_evidence['dnm'] = [dnm_prob if x in dnm['Genes'].to_list() \
                                  else (1 - dnm_prob) \
                         for x in extra_evidence.index]

## DE data 
## note that extra_evidence df has same row order as candidate_genes
## add column of ENSG IDs to make merge possible with dexpr
extra_evidence.reset_index(drop = True, inplace = True)  ## won't work unless gene symbol index is dropped
extra_evidence['Name'] = candidate_genes['Name']

## merge with dexpr
extra_evidence = extra_evidence.merge(dexpr[['genes','P.Value']], \
                                      how = 'left', left_on = 'Name', right_on = 'genes')
## clean-up dataframe
extra_evidence.drop(labels = ['Name', 'genes'], axis = 'columns', inplace = True)
extra_evidence.rename(columns={'P.Value': 'DE'}, inplace = True)
extra_evidence.head()

,DTS,cap4_enhancer_no,fantom5_enhancer_no,cp_enhancer_no,gz_enhancer_no,dnm,DE
0,0.922759,0.006389,0.527739,0.280968,0.325234,0.9666,0.086510
1,0.919416,0.888584,0.426653,0.427392,0.279037,0.9666,NaN
2,0.919311,0.010999,0.519295,0.434185,0.290770,0.9666,NaN
3,0.907090,0.784964,0.309512,0.515902,0.357274,0.9666,0.006472
4,0.890168,0.172973,0.474665,0.602046,0.444174,0.9666,NaN


Missing DE p-values should be replaced with the median p-value, rather than giving it a p-value of 0 (aka significant). 

In [43]:
## 1261 genes have DE info, 376 have missing values 
extra_evidence['DE'].count()  ## method to get number of non-missing values in the column
print("The median DE p-value is", extra_evidence['DE'].median(skipna = True))

## replace missing values with median
extra_evidence['DE'].fillna(extra_evidence['DE'].median(skipna = True), inplace = True)

1261

The median DE p-value is 0.3433


In [44]:
extra_evidence.head()

,DTS,cap4_enhancer_no,fantom5_enhancer_no,cp_enhancer_no,gz_enhancer_no,dnm,DE
0,0.922759,0.006389,0.527739,0.280968,0.325234,0.9666,0.086510
1,0.919416,0.888584,0.426653,0.427392,0.279037,0.9666,0.343300
2,0.919311,0.010999,0.519295,0.434185,0.290770,0.9666,0.343300
3,0.907090,0.784964,0.309512,0.515902,0.357274,0.9666,0.006472
4,0.890168,0.172973,0.474665,0.602046,0.444174,0.9666,0.343300


### Fisher's product method of combining data  
See: https://en.wikipedia.org/wiki/Fisher%27s_method

When individual p-values are small, the meta-test-statistic should be large. When all individual null hypotheses are true and their test stats/p-values are independent, this meta-test-statistic has a chi-squared dist with 2k degrees of freedom (k = number of tests that are combined).  

The meta p-values are then calculated using what the paper's supplement (pgs. 2-3) describes as the "inverse cumulative distribution function (CDF) of chi-squared (chi2) distribution." In practice, this is 1 - (chi2's CDF). In Python, this is found using scipy's survival function (sf). See methods in https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.chi2.html

**The final values here are used as surrogates for the Bayesian factor!** This is explained in the Supplementary Note for the iRIGS paper.     

In [46]:
## From Fisher's method: chi-squared dist has degrees of freedom = 2 * number of tests  
fishers_chi_df = 2 * len(extra_evidence.columns)

## np.log default IS A NATURAL LOG 
## this is the meta-test-statistic. sum is across the row 
fishers_combo = extra_evidence.apply(lambda x: -2 * np.sum(np.log(x)), axis = 1)  
## this is a pandas Series object

## get p-values of the meta-test-statistic:  
fishers_combo = stats.chi2.sf(fishers_combo, df = fishers_chi_df)

## get negative log of all these weights. I think this is so larger weights -> 
##     the gene is more likely to be important for the SNP GWAS signal. 
fishers_combo = -np.log(fishers_combo)

In [47]:
candidate_genes = candidate_genes.assign(transformed_Fisherprod = fishers_combo)
candidate_genes

,SNP,SNP_chr,SNP_pos_hg19,Name,Description,chrom,type,start_hg19,end_hg19,strand,...,gene_status,loci_level,alias_symbol,official_name,DTS,cp_enhancer_no,gz_enhancer_no,cap4_enhancer_no,fantom5_enhancer_no,transformed_Fisherprod
0,rs115329265,chr6,28712247,ENSG00000185130,HIST1H2BL,chr6,gene,27775257,27775709,-,...,KNOWN,2,HIST1H2BL,HIST1H2BL,936538,10,10,223.0,4,2.360891
1,rs115329265,chr6,28712247,ENSG00000182611,HIST1H2AJ,chr6,gene,27782112,27782607,-,...,KNOWN,2,HIST1H2AJ,HIST1H2AJ,929640,8,9,0.0,4,0.153445
2,rs115329265,chr6,28712247,ENSG00000196374,HIST1H2BM,chr6,gene,27782822,27783267,+,...,KNOWN,2,HIST1H2BM,HIST1H2BM,929425,8,9,211.0,4,1.329594
3,rs115329265,chr6,28712247,ENSG00000233822,HIST1H2BN,chr6,gene,27806323,27819974,+,...,KNOWN,2,HIST1H2BN,HIST1H2BN,905924,6,7,26.0,6,1.270672
4,rs115329265,chr6,28712247,ENSG00000184357,HIST1H1B,chr6,gene,27834570,27835359,-,...,KNOWN,2,HIST1H1B,HIST1H1B,876888,4,5,130.0,4,0.278950
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1632,chr5_140143664_I,chr5,140143664,ENSG00000178913,TAF7,chr5,gene,140698057,140700330,-,...,KNOWN,2,TAF7,TAF7,556666,3,4,84.0,2,0.044525
1633,chr5_140143664_I,chr5,140143664,ENSG00000131504,DIAPH1,chr5,gene,140894583,140998622,-,...,KNOWN,2,DIAPH1,DIAPH1,854958,30,25,186.0,8,5.518480
1634,chr5_140143664_I,chr5,140143664,ENSG00000171720,HDAC3,chr5,gene,141000443,141016437,-,...,KNOWN,2,HDAC3,HDAC3,872773,1,0,83.0,0,0.008950
1635,chr5_140143664_I,chr5,140143664,ENSG00000120318,ARAP3,chr5,gene,141013251,141061788,-,...,KNOWN,1,ARAP3,ARAP3,918124,16,20,69.0,2,0.824276


**Optional code chunk: not required to run**

I can explore what's going on with Fisher's method and check my manual calculations with the scipy stats function: https://docs.scipy.org/doc/scipy-0.16.0/reference/generated/scipy.stats.combine_pvalues.html 

In [48]:
## comparing the manual Fisher's method above with the scipy function for Fisher's method
# fishers_combo[0:10] 

# list_fisherscombo = []
# for ele in extra_evidence.values:
#     list_fisherscombo.append(stats.combine_pvalues(pvalues = ele, method = 'fisher'))

# for (test, value) in list_fisherscombo[0:10]:
#     print(-np.log(value))

## what had a very high surrogate Bayes factor? = a likely risk gene 
# np.argmax(fishers_combo)
# candidate_genes.loc[1493,]
# ## there were a lot of enhancers found and a smaller distance, surrogate Bayes factor is ~ 66.9
# extra_evidence.loc[1493,]
# ## one of the individual p-values is very low
# stats.combine_pvalues(pvalues = extra_evidence.loc[1493,], method = 'fisher')
# ## test stat is large

# ## what had a very low surrogate Bayes factor? = not likely risk gene
# np.argmin(fishers_combo)
# candidate_genes.loc[1037,]
# ## almost no enhancers found, distance from SNP is far, surrogate Bayes factor is ~ 0.0007 
# extra_evidence.loc[1037,]
# ## all individual p-values are high
# stats.combine_pvalues(pvalues = extra_evidence.loc[1037,], method = 'fisher')
# ## test stat is small

# ## another with a very high surrogate Bayes factor
np.where(fishers_combo >= 50)
candidate_genes.loc[126,]
## close distance, most enhancer numbers are high -> surrogate Bayes factor ~ 52.6
extra_evidence.loc[126,]
## two p-values are very small  
stats.combine_pvalues(pvalues = extra_evidence.loc[126,], method = 'fisher')
## test stat is high

(array([ 126, 1490, 1493]),)

SNP                          chr7_2025096_I
SNP_chr                                chr7
SNP_pos_hg19                        2025096
Name                        ENSG00000002822
Description                          MAD1L1
chrom                                  chr7
type                                   gene
start_hg19                          1855383
end_hg19                            2272878
strand                                    -
ts_id                     ENSG00000002822.9
gene_type                    protein_coding
gene_status                           KNOWN
loci_level                                1
alias_symbol                         MAD1L1
official_name                        MAD1L1
DTS                                  247782
cp_enhancer_no                           80
gz_enhancer_no                           76
cap4_enhancer_no                        112
fantom5_enhancer_no                       1
transformed_Fisherprod              52.6165
Name: 126, dtype: object

DTS                    1.991888e-01
cap4_enhancer_no       2.714492e-01
fantom5_enhancer_no    7.579500e-01
cp_enhancer_no         1.472854e-18
gz_enhancer_no         5.766056e-12
dnm                    9.666000e-01
DE                     2.026000e-01
Name: 126, dtype: float64

(143.52694878077799, 1.4091035662651953e-23)

## Gibbs sampling: burn-in step
**CHANGES from original code:**

* Gibbs sampling is not done on SNP loci with only one candidate gene. This is a change from the original code, which would add a sample identical to the previous sample.  
* Python version counts genes after every Gibbs sample in a table column. The R version originally stored rows of strings (genes) for every sample and counted genes only after a full round through SNP loci finished. 

**Thoughts:** 
* I think we work with square-roots of the threshold and sum of squared differences, so (1) sign/direction of difference won't matter with squaring and (2) we avoid dealing with very small numbers (loss of precision). This way of doing things was in the original R code.    
* should SNPs with only one candidate gene be removed completely from the candidate gene space for Gibbs sampling? They will bias results because they are always used to calculate the network factors for other loci candidates - however, this may be what we want if we suspect that these candidate genes are highly likely to be disease risk genes.
* what should happen in the unlikely scenario that all candidate genes for a SNP are also candidates for other loci (so there will be an empty table for calculating the network factor)? Perhaps we would need to go forward with a new random sample? Or would this "never" happen because users will not choose SNPs close together?
* Would Python dictionaries (SNPs to genes) of dictionaries (each gene has a count) be a good data structure to use? 

[How to time chunks of code](https://stackoverflow.com/questions/2866380/how-can-i-time-a-code-segment-for-testing-performance-with-pythons-timeit#2866456)

### Setup

In [51]:
## flag to remove candidate genes (rows) from network/columns calculations 
flag_exclude_sharedrowcols = True  

## square-root of the threshold we want! stopping criterion
stopping_threshold = math.sqrt(10**(-4))
dif = stopping_threshold + 1  ## used for stopping criterion

## round = a set of sampling thru all loci
burnin_rounds = 0  ## counter for number of rounds 
max_burnin_rounds = 3000  ## maximum number of rounds allowed at burn-in step

## used to record counts of SNP-gene pairs from all previous rounds
burnin_gene_counts = candidate_genes[['SNP', 'alias_symbol']].copy()

## get SNP loci with only one candidate gene: no need to use Gibbs sampling for these loci
##     to find the most probable candidate
## however, this locus is included in Gibbs sampling so it can be used to calculate the network factor
SNPs_one_candidate = burnin_gene_counts.copy().groupby('SNP', sort = False).count()
SNPs_one_candidate = set(SNPs_one_candidate[SNPs_one_candidate['alias_symbol'] == 1].index.to_list())

## starting set: 1 per SNP locus 
## another way to do this: random sort and then grab first row for each SNP: 
##       https://stackoverflow.com/questions/36390406/sample-each-group-after-pandas-groupby
current_geneset = burnin_gene_counts.copy().groupby('SNP', sort = False).apply(pd.DataFrame.sample, n=1)
current_geneset

## used to iterate through SNPs in the same order as current_gene_set's rows
SNP_loci = burnin_gene_counts['SNP'].unique()  ## this is a numpy array

## use key for fast updating of counts: 
##     using the dataframe index was messy because the current_geneset wouldn't have all the index values
burnin_gene_counts['key'] = burnin_gene_counts['SNP'] + "," + burnin_gene_counts['alias_symbol']

## set up counter for candidate genes
burnin_gene_counts['counts'] = 0
burnin_gene_counts

## how many SNP loci are sampled per round
SNPs_per_round = len(SNP_loci) - len(SNPs_one_candidate)
SNPs_per_round

,,SNP,alias_symbol
SNP,,,
rs115329265,20,rs115329265,OR2B3
rs1702294,37,rs1702294,DPYD
rs11191419,59,rs11191419,CNNM2
rs2007044,83,rs2007044,TULP3
rs4129585,89,rs4129585,TSNARE1
...,...,...,...
rs4240748,1500,rs4240748,CLLU1
rs2909457,1507,rs2909457,DPP4
rs56873913,1552,rs56873913,DKKL1


,SNP,alias_symbol,key,counts
0,rs115329265,HIST1H2BL,"rs115329265,HIST1H2BL",0
1,rs115329265,HIST1H2AJ,"rs115329265,HIST1H2AJ",0
2,rs115329265,HIST1H2BM,"rs115329265,HIST1H2BM",0
3,rs115329265,HIST1H2BN,"rs115329265,HIST1H2BN",0
4,rs115329265,HIST1H1B,"rs115329265,HIST1H1B",0
...,...,...,...,...
1632,chr5_140143664_I,TAF7,"chr5_140143664_I,TAF7",0
1633,chr5_140143664_I,DIAPH1,"chr5_140143664_I,DIAPH1",0
1634,chr5_140143664_I,HDAC3,"chr5_140143664_I,HDAC3",0
1635,chr5_140143664_I,ARAP3,"chr5_140143664_I,ARAP3",0


101

### Burn-in Loop

Starting with the second round of Gibbs sampling, it prints difference from previous round. 

It took me 738 seconds (12.3 minutes) to do 619 rounds to reach a difference below threshold. 

In [52]:
t0 = time.time()  ## used to record time in system

## outer loop: checking dif in gene sampling probabilities between previous two rounds
##                and checking number of rounds 
while ((dif > stopping_threshold) and (burnin_rounds < max_burnin_rounds)):
    ## run BEFORE iterating through genes for a round. 
    ## store count of genes in Gibbs samples for this round
    burnin_gene_counts['round_ct'] = 0
    
    ## inner loop (the round): for each SNP loci...
    for i in SNP_loci:  
        ## note that if there is only one candidate gene for the SNP locus, 
        ##      the chunk doesn't run and the locus is SKIPPED (doesn't go thru sampling process)
        if i not in SNPs_one_candidate:
            ## grab Bayesian factor surrogate for each candidate gene in the current SNP locus i 
            currentloci_genes = candidate_genes[candidate_genes['SNP'] == i].filter( \
                                    items = ['alias_symbol', 'transformed_Fisherprod'])

            ## look at the genes picked in other loci (not i)   
            otherloci_genes = current_geneset[current_geneset['SNP'] != i]
            ## take out genes that are also in SNP locus i 
            if flag_exclude_sharedrowcols is True:  
                otherloci_genes = otherloci_genes[~ otherloci_genes['alias_symbol'].isin(\
                                                        currentloci_genes['alias_symbol'])]
                ## if this takes out all of the genes, this would cause issues downstream. 
                ##      I haven't had this flag go off before. Perhaps can be dealt with by drawing a 
                ##      new current_geneset sample (like what is done at the beginning). 
                if len(otherloci_genes) == 0:
                    raise ValueError(\
                        'Error: too much overlap between the gene set {genelist} and the SNP locus {SNP} led to issues calculating the network factor'.format(\
                        genelist = current_geneset['alias_symbol'].to_list(), SNP = i)) 

            ## retrieve network info: rows are candidate genes from SNP locus i, 
            ##                        columns are genes from other SNP loci (not i)
            networkinfo = network_matrix.loc[currentloci_genes['alias_symbol'].to_list(), \
                                             otherloci_genes['alias_symbol'].to_list()]
            network_factors = networkinfo.sum(axis = 'columns')  ## this is a pandas series 
            ## it is okay to do this because the series has the same row order as currentloci_genes
            currentloci_genes['network_factor'] = network_factors.to_list()

            ## calculate probability weighting of candidate genes using network factor and surrogate Bayesian factor     
            currentloci_genes['probweights'] = \
                currentloci_genes['transformed_Fisherprod'] * currentloci_genes['network_factor']
            ## if all the calculated weights are zero, set uniform prob dist 
            if currentloci_genes['probweights'].sum() == 0:
                currentloci_genes['probweights'] = 1

            ## choose loci's new state using the probability dist 
            newstate = currentloci_genes.sample(n = 1, axis = 0, weights = 'probweights')['alias_symbol'].to_list()
            ## in Gibbs sample geneset, find row for SNP locus i, replace the entry with the new state
            current_geneset.loc[i, 'alias_symbol'] = newstate    

            ## record the current_geneset (Gibbs sample) in the round_ct column 
            ## using a for loop here with itertuples is ~30x slower 
            current_geneset['key'] = current_geneset['SNP'] + "," + current_geneset['alias_symbol']            
            burnin_gene_counts.loc[burnin_gene_counts['key'].isin(current_geneset['key']), 'round_ct'] += 1
            
            ## no garbage collection here because it increases time for a loop by ~7x
        ## end the if statement for only Gibbs sampling on SNP loci with > 1 candidate gene 
    ## end for loop of iterating thru each SNP loci    

    ## a round is finished so increment this for stopping criteria 
    burnin_rounds += 1
    if ((burnin_rounds % 50) == 0):
        print("Burn-in sampling, finished full round thru SNP loci:", burnin_rounds, "\n")

    ## add together the previous counts and new round_ct -> get new sampling counts
    burnin_gene_counts['round_ct'] += burnin_gene_counts['counts'] 

    ## starting with the second round, calculate dif between sampling probabilities 
    ##    between previous round and current round
    if burnin_rounds > 1:
        ## sampling frequencies from previous round
        burnin_gene_counts['freq_previous'] = burnin_gene_counts['counts'] / ((burnin_rounds - 1) * SNPs_per_round)
        ## sampling frequencies from current round
        burnin_gene_counts['freq_now'] = burnin_gene_counts['round_ct'] / (burnin_rounds * SNPs_per_round)
        ## calculate sum of squared differences, then square-root this sum. 
        dif = math.sqrt(pd.Series.sum((burnin_gene_counts['freq_now'] - burnin_gene_counts['freq_previous']) ** 2))
        print("dif is", dif, "\n")

    ## replace old counts with current counts. 
    burnin_gene_counts['counts'] = burnin_gene_counts['round_ct']
    ## drop freq columns if they're there
    burnin_gene_counts = burnin_gene_counts[['SNP', 'alias_symbol', 'counts', 'key']]  

## print the time it took to run
t1 = time.time()
burnin_time = t1 - t0
burnin_time

print("Number of burn-in rounds of Gibbs sampling:", burnin_rounds)

dif is 4.602453708534399 

dif is 2.6366611143350505 

dif is 1.7967273745811914 

dif is 1.4965673234283758 

dif is 1.1565741850795737 

dif is 0.9401945199956643 

dif is 0.8572687036211708 

dif is 0.7288495283337117 

dif is 0.6746137230919929 

dif is 0.6213502768749306 

dif is 0.5870879776592086 

dif is 0.5553887625666095 

dif is 0.4839013426032615 

dif is 0.43745207841980227 

dif is 0.40866478644892146 

dif is 0.35881022128570367 

dif is 0.3761855464144371 

dif is 0.35153463660438916 

dif is 0.33184940941941254 

dif is 0.3125238106728691 

dif is 0.29711965041905997 

dif is 0.2905986410367301 

dif is 0.2836970824377936 

dif is 0.2547773889770668 

dif is 0.2549454467288552 

dif is 0.23911978693840166 

dif is 0.2418581162166099 

dif is 0.22712233106312585 

dif is 0.21977091363272236 

dif is 0.2209989456180215 

dif is 0.21256829386221351 

dif is 0.19099499314331592 

dif is 0.19915729892938983 

dif is 0.18179231203037274 

dif is 0.17997660660069564 

dif is 

dif is 0.023681718047293136 

dif is 0.022426384646559227 

dif is 0.023818966171720145 

dif is 0.02297046307450034 

dif is 0.023778745689652277 

dif is 0.022655431881396085 

dif is 0.02374775037854777 

dif is 0.0243890671564934 

dif is 0.023281597179218187 

dif is 0.023270223798417946 

dif is 0.02341545000616697 

dif is 0.02325490860949931 

dif is 0.021873608348011102 

dif is 0.022662350566789485 

dif is 0.02350958543662739 

dif is 0.022695415419817004 

dif is 0.023010640679894397 

dif is 0.022046079840381465 

dif is 0.023191939612313212 

dif is 0.022147371050219564 

dif is 0.022560358727873337 

dif is 0.02181389673229712 

dif is 0.02245706700630412 

dif is 0.021911298859934335 

dif is 0.021569160610807214 

dif is 0.02252282639434814 

Burn-in sampling, finished full round thru SNP loci: 300 

dif is 0.02201570333901054 

dif is 0.02153029380710425 

dif is 0.021740667371673014 

dif is 0.020962901400233255 

dif is 0.02180085661741982 

dif is 0.020285003000759

dif is 0.01193366730661319 

dif is 0.012057267729588613 

dif is 0.012007958978953293 

dif is 0.012010016846908 

dif is 0.01278181759348681 

dif is 0.011332240446553279 

dif is 0.012067519734205147 

dif is 0.012317277911723637 

dif is 0.011319815180858736 

dif is 0.011804437979910636 

Burn-in sampling, finished full round thru SNP loci: 550 

dif is 0.011944807991391557 

dif is 0.011500089176719864 

dif is 0.011251723353912668 

dif is 0.011032914469402839 

dif is 0.011565138950909038 

dif is 0.011584472441211283 

dif is 0.011650419148028836 

dif is 0.0118401915828492 

dif is 0.012063877004068486 

dif is 0.011015585429401098 

dif is 0.011120506708787167 

dif is 0.011787781789737814 

dif is 0.011020230085783478 

dif is 0.011439012168974227 

dif is 0.011042428150757993 

dif is 0.01148537885541956 

dif is 0.012008551543135368 

dif is 0.011376640527464808 

dif is 0.01164186915607843 

dif is 0.010710687400476421 

dif is 0.011017354807321302 

dif is 0.01133477152

706.4467251300812

Number of burn-in rounds of Gibbs sampling: 583


## Gibbs sampling: post-burn-in

Keep the current_geneset from the last burn-in round. Start over the counting of genes sampled - these proportions should now approximate the posterior probability. 

**same changes and thoughts noted above for burn-in step**

### Setup

In [53]:
## keep these variables: current_geneset from last burn-in round, 
##              SNP_loci, SNPs_one_candidate, SNPs_per_round

## round = a set of sampling thru all loci
max_final_rounds = 3000    ### maximum sampling round allowed at post burn in step

## reset variables
dif = stopping_threshold + 1  ## used for stopping criterion
final_rounds = 0  ## counter for number of rounds thru all SNP loci

## used to record counts of SNP-gene pairs from all previous rounds
final_gene_counts = burnin_gene_counts.copy()
## set up counter for candidate genes
final_gene_counts['counts'] = 0

final_gene_counts

,SNP,alias_symbol,counts,key
0,rs115329265,HIST1H2BL,0,"rs115329265,HIST1H2BL"
1,rs115329265,HIST1H2AJ,0,"rs115329265,HIST1H2AJ"
2,rs115329265,HIST1H2BM,0,"rs115329265,HIST1H2BM"
3,rs115329265,HIST1H2BN,0,"rs115329265,HIST1H2BN"
4,rs115329265,HIST1H1B,0,"rs115329265,HIST1H1B"
...,...,...,...,...
1632,chr5_140143664_I,TAF7,0,"chr5_140143664_I,TAF7"
1633,chr5_140143664_I,DIAPH1,0,"chr5_140143664_I,DIAPH1"
1634,chr5_140143664_I,HDAC3,0,"chr5_140143664_I,HDAC3"
1635,chr5_140143664_I,ARAP3,0,"chr5_140143664_I,ARAP3"


### Post-Burn-in Loop

Starting with the second round of Gibbs sampling, it prints difference from previous round. 

It took me 766 seconds (12.8 minutes) to do 610 rounds to reach a difference below threshold. 

When I ran this for a second time for two post-burn-in loops (running both the setup code chunk and the following code chunk), the second time took me 807 seconds (13.45 min) to do 638 rounds. 

In [54]:
t0 = time.time()  ## used to record time in system

## outer loop: checking dif in gene sampling probabilities between previous two rounds
##                and checking number of rounds 
while ((dif > stopping_threshold) and (final_rounds < max_final_rounds)):
    ## run BEFORE iterating through genes for a round. 
    ## store count of genes in Gibbs samples for this round
    final_gene_counts['round_ct'] = 0
    
    ## inner loop (the round): for each SNP loci...    
    for i in SNP_loci: 
        ## note that if there is only one candidate gene for the SNP locus, 
        ##      the chunk doesn't run and the locus is SKIPPED (doesn't go thru sampling process)
        if i not in SNPs_one_candidate:
            ## grab Bayesian factor surrogate for each candidate gene in the current SNP locus i 
            currentloci_genes = candidate_genes[candidate_genes['SNP'] == i].filter( \
                                    items = ['alias_symbol', 'transformed_Fisherprod'])

            ## look at the genes picked in other loci (not i)    
            otherloci_genes = current_geneset[current_geneset['SNP'] != i]
            ## take out genes that are also in SNP locus i 
            if flag_exclude_sharedrowcols is True:  
                otherloci_genes = otherloci_genes[~ otherloci_genes['alias_symbol'].isin(\
                                                        currentloci_genes['alias_symbol'])]
                ## if this takes out all of the genes, this would cause issues downstream. 
                ##      I haven't had this flag go off before. Perhaps can be dealt with by drawing a 
                ##      new current_geneset sample (like what is done at the beginning). 
                if len(otherloci_genes) == 0:
                    raise ValueError(\
                        'Error: too much overlap between the gene set {genelist} and the SNP locus {SNP} led to issues calculating the network factor'.format(\
                        genelist = current_geneset['alias_symbol'].to_list(), SNP = i)) 

            ## retrieve network info: rows are candidate genes from SNP locus i, 
            ##                        columns are genes from other SNP loci (not i)
            networkinfo = network_matrix.loc[currentloci_genes['alias_symbol'].to_list(), \
                                             otherloci_genes['alias_symbol'].to_list()]
            network_factors = networkinfo.sum(axis = 'columns')  ## this is a pandas series 
            ## it is okay to do this because the series has the same row order as currentloci_genes
            currentloci_genes['network_factor'] = network_factors.to_list()

            ## calculate probability weighting of candidate genes using network factor and surrogate Bayesian factor     
            currentloci_genes['probweights'] = currentloci_genes['transformed_Fisherprod'] * currentloci_genes['network_factor']
            ## if all the calculated weights are zero, set uniform prob dist 
            if currentloci_genes['probweights'].sum() == 0:
                currentloci_genes['probweights'] = 1

            ## choose loci's new state using the probability dist 
            newstate = currentloci_genes.sample(n = 1, axis = 0, weights = 'probweights')['alias_symbol'].to_list()
            ## in Gibbs sample geneset, find row for SNP locus i, replace the entry with the new state
            current_geneset.loc[i, 'alias_symbol'] = newstate    

            ## record the current_geneset (Gibbs sample) in the round_ct column 
            ## using a for loop here with itertuples is ~30x slower 
            current_geneset['key'] = current_geneset['SNP'] + "," + current_geneset['alias_symbol']
            final_gene_counts.loc[final_gene_counts['key'].isin(current_geneset['key']), 'round_ct'] += 1
            
            ## no garbage collection here because it increases time for a loop by ~7x
        ## end the if statement for only Gibbs sampling on SNP loci with > 1 candidate gene 
    ## end for loop of iterating thru each SNP loci    

    ## a round is finished so increment this for stopping criteria 
    final_rounds += 1
    if ((final_rounds % 50) == 0):
        print("Final (post-burn-in) sampling, finished full round thru SNP loci:", final_rounds, "\n")

    ## add together the previous counts and new round_ct -> get new sampling counts
    final_gene_counts['round_ct'] += final_gene_counts['counts'] 

    ## starting with the second round, calculate dif
    ##     between sampling probabilities from previous round and current round
    if final_rounds > 1:
        ## sampling frequencies from previous round
        final_gene_counts['freq_previous'] = final_gene_counts['counts'] / ((final_rounds - 1) * SNPs_per_round)
        ## sampling frequencies from current round
        final_gene_counts['freq_now'] = final_gene_counts['round_ct'] / (final_rounds * SNPs_per_round)
        ## calculate sum of squared differences, then square-root this sum. 
        dif = math.sqrt(pd.Series.sum((final_gene_counts['freq_now'] - final_gene_counts['freq_previous']) ** 2))
        print("dif is", dif, "\n")

    ## replace old counts with current counts. 
    final_gene_counts['counts'] = final_gene_counts['round_ct']
    ## drop freq columns if they're there
    final_gene_counts = final_gene_counts[['SNP', 'alias_symbol', 'counts', 'key']]  

## print the time it took to run
t1 = time.time()
final_time = t1 - t0
final_time

print("Number of final (post-burn-in) rounds of Gibbs sampling:", final_rounds)

dif is 4.104327635875756 

dif is 2.6409136989475455 

dif is 1.745411294409637 

dif is 1.4861547746167838 

dif is 1.1230761730199248 

dif is 0.9758704043080402 

dif is 0.8093166070814872 

dif is 0.7542798865491305 

dif is 0.690340621772406 

dif is 0.5858124893024745 

dif is 0.5413065636763034 

dif is 0.48604819561765816 

dif is 0.46951089898835446 

dif is 0.44428858394265713 

dif is 0.4183222020082175 

dif is 0.38438547894377495 

dif is 0.3794956567020391 

dif is 0.34581080114575313 

dif is 0.32258005735979145 

dif is 0.3265351924849056 

dif is 0.2894390330395978 

dif is 0.28518418205602625 

dif is 0.26417674803458635 

dif is 0.2606288608094724 

dif is 0.25289490245631446 

dif is 0.24789586206143413 

dif is 0.23371959166274106 

dif is 0.24000409383222895 

dif is 0.22330144712538105 

dif is 0.2102238258457328 

dif is 0.20371332895842756 

dif is 0.20452520863533025 

dif is 0.19124360036902868 

dif is 0.19145775965701478 

dif is 0.18021643489614703 

dif i

dif is 0.024383310043854828 

dif is 0.024794494627562086 

dif is 0.023087208326457016 

dif is 0.0223790602800941 

dif is 0.024123707561516328 

dif is 0.023884925203878676 

dif is 0.02437537120392872 

dif is 0.02266806261168412 

dif is 0.023130706446494485 

dif is 0.022875561894457774 

dif is 0.023808006929003814 

dif is 0.0230941097131435 

dif is 0.023007376966996264 

dif is 0.022243353371990973 

dif is 0.02337731624061857 

dif is 0.02356692559625294 

dif is 0.022884620244764155 

dif is 0.022181411625549683 

dif is 0.022362724281247255 

dif is 0.02287508421746003 

dif is 0.022615579869567207 

dif is 0.022156210907674827 

dif is 0.02211890666304245 

dif is 0.021368173616553292 

dif is 0.022162596197695332 

dif is 0.02285919599843703 

dif is 0.022046573407137626 

dif is 0.021549248360725207 

dif is 0.021066641665357422 

Final (post-burn-in) sampling, finished full round thru SNP loci: 300 

dif is 0.022568976416755242 

dif is 0.02213581543390596 

dif is 0.0

dif is 0.012121833894403907 

dif is 0.01237976726447999 

dif is 0.011879016399236133 

dif is 0.011759740198140025 

dif is 0.01194430576827477 

dif is 0.012297341602868475 

dif is 0.012190901927987077 

dif is 0.011483977289683912 

dif is 0.012177829373136355 

dif is 0.011915067137766962 

dif is 0.011987546577340172 

dif is 0.01205739416760665 

dif is 0.011984386726753806 

dif is 0.011719707867733168 

dif is 0.011070173302404895 

Final (post-burn-in) sampling, finished full round thru SNP loci: 550 

dif is 0.011531184429542573 

dif is 0.011734118468568894 

dif is 0.011692072479046314 

dif is 0.011777311983586657 

dif is 0.011849189296858258 

dif is 0.011591228255443287 

dif is 0.011128174142941018 

dif is 0.011925084539346346 

dif is 0.012123720300576261 

dif is 0.010962567338860151 

dif is 0.01179684385245157 

dif is 0.011407802799371834 

dif is 0.011035646638847398 

dif is 0.011060412011670436 

dif is 0.0116919522448791 

dif is 0.011338700004516221 

dif 

736.2830719947815

Number of final (post-burn-in) rounds of Gibbs sampling: 621


## Output
Will write files and display the final dataframes.    

**CHANGES from original code:**  
* Reproducing the *Nature Methods* paper's Supplementary Table 1 as output  

**Parameters:**
* path for result files directory   
* path for original type of result file    
* path for result file of the same format as Supplementary Table 1 from the paper   

**Remember to change these last two parameters between runs if you don't want to rewrite your files from previous runs.**   

In [55]:
## before running, check that these paths are what you want them to be. 
##     Especially when you don't want to rewrite files from previous runs! 
results_dir_path = Path.cwd().joinpath('iRIGS_results')
original_output_path = Path.cwd().joinpath('iRIGS_results', 'SCZ_trial2_original.txt')
supplement_table_path = Path.cwd().joinpath('iRIGS_results', 'SCZ_trial2_supple1.txt')

In [56]:
print("Processing and recording the final sampling probabilities!\n")

## calculate final sampling probabilities 
final_gene_counts['posterior_probability'] = final_gene_counts['counts'] / (final_rounds * SNPs_per_round)

## final dataframe
final_gene_counts = final_gene_counts.filter(items = ['SNP', 'alias_symbol', 'posterior_probability'])
final_gene_counts.rename(columns = {'alias_symbol':'candidate_gene_symbol'}, inplace = True)
final_gene_counts.sort_values(by = ['SNP', 'posterior_probability'], ascending = [True, False], \
                              ignore_index = True, inplace = True)

## create directory for result files
if results_dir_path.exists() is False:
    results_dir_path.mkdir(parents = True)
    
## create original result file
final_gene_counts.to_csv(original_output_path, sep = '\t', index = False)

## Build table like in Supplementary Table 1 of paper
## count number of candidate genes per SNP locus
supple1 = final_gene_counts.copy().groupby('SNP').head(1)

SNPs_per_loci = final_gene_counts.copy().groupby('SNP').size()  ## this is a series with SNP as index
SNPs_per_loci.name = 'number_of_candidates'
SNPs_per_loci = SNPs_per_loci.reset_index()

supple1 = supple1.merge(SNPs_per_loci, on = 'SNP')

## reorder to match supplementary table 
supple1 = supple1[['candidate_gene_symbol', 'posterior_probability', 'SNP', 'number_of_candidates']]

## create supplementary table 1 result file
supple1.to_csv(supplement_table_path, sep = '\t', index = False)

Processing and recording the final sampling probabilities!

